In [1]:
#desktopで動かす。
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

import tidalUtl.PrpUtl as prp

In [2]:
#import sys
#sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

https://www.kaggle.com/tidalryoku/new-baseline-pytorch-moa/

# Version

__ver1__<br>
baseline：CV:0.01465 LB:0.01874<br>
__ver2__<br>
Hyperopt, 2Layer：CV:0.01460 LB:0.01869<br>
__ver3__<br>
3Layer：CV:0.01464 LB:LB:0.01868<br>


# Config

In [3]:
INPUT = "/home/tidal/ML_Data/MoA/lish-moa"
OUTPUT = "/home/tidal/ML_Data/MoA/output"
#INPUT = "/Users/hfuis/ML_Data/MoA/lish-moa"
#OUTPUT = "/Users/hfuis/ML_Data/MoA/output"

SUBMIT = OUTPUT + "/submittion/"
SAVEMODEL = OUTPUT + "/modelPyTorch/"

In [4]:
#Loading
trainFeature = pd.read_csv(INPUT + '/train_features.csv')
testFeature = pd.read_csv(INPUT + '/test_features.csv')
trainTargetScored = pd.read_csv(INPUT + '/train_targets_scored.csv')
sample_submission = pd.read_csv(INPUT + '/sample_submission.csv')

In [5]:
GENES = [col for col in trainFeature.columns if col.startswith('g-')] #gから始まる列名のセット
CELLS = [col for col in trainFeature.columns if col.startswith('c-')] #cから始まる列名のセット

In [6]:
#Seed固定
def seed_everything(seed=42):
    #data取得についてのランダム性固定
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #cudnnによる演算の安定化(評価値の安定)
    torch.backends.cudnn.deterministic = True
    
    #os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(seed=42)

# Preprocessing

## Func: In & Out Type is DataFrame

### PCA features add

In [7]:
# GENES
n_comp = 50

inTrain = trainFeature[GENES]
inTest = testFeature[GENES]

#PCA実行＆変換後のデータ作成
pca_train, pca_test, _ = prp.tidalPCA(inTrain, inTest, Dim=n_comp, random_state=42)

#columの名前付け
trainTmp = pd.DataFrame(pca_train, columns=[f'pca_G-{i}' for i in range(n_comp)])
testTmp = pd.DataFrame(pca_test, columns=[f'pca_G-{i}' for i in range(n_comp)])

#データに付け足し
trainFeature = pd.concat((trainFeature, trainTmp), axis=1)
testFeature = pd.concat((testFeature, testTmp), axis=1)

In [8]:
# CELLS
# CELLSもGENESと同様。
n_comp = 15

inTrain = trainFeature[CELLS]
inTest = testFeature[CELLS]

pca_train, pca_test, _ = prp.tidalPCA(inTrain, inTest, Dim=n_comp, random_state=42)

trainTmp = pd.DataFrame(pca_train, columns=[f'pca_C-{i}' for i in range(n_comp)])
testTmp = pd.DataFrame(pca_test, columns=[f'pca_C-{i}' for i in range(n_comp)])

trainFeature = pd.concat((trainFeature, trainTmp), axis=1)
testFeature = pd.concat((testFeature, testTmp), axis=1)

### feature Selection using Variance Encoding

In [9]:
data = trainFeature.append(testFeature)

#['sig_id','cp_type','cp_time','cp_dose']を除いたfeatureで低い分散の特徴量を除去
data_transformed = prp.tidalVarianceThrs(data.iloc[:, 4:], threshold=0.5)


trainFeature_transformed = data_transformed[ : trainFeature.shape[0]]
testFeature_transformed = data_transformed[-testFeature.shape[0] : ]

trainFeature = trainFeature[['sig_id','cp_type','cp_time','cp_dose']]
trainFeature = pd.concat([trainFeature, pd.DataFrame(trainFeature_transformed)], axis=1)

testFeature = testFeature[['sig_id','cp_type','cp_time','cp_dose']]
testFeature = pd.concat([testFeature, pd.DataFrame(testFeature_transformed)], axis=1)

## Collecting(モデルに突っ込むデータ形式に整形)

### cp_type = ctl_vehicleのレコードを削除

__※提出用データ(test)も同様に一部レコードを削除するが、こちらは最後submittionデータを作る際に0埋めを行う。__

In [10]:
#Pkey(sig_id)でfeatureとtargetを内部結合。
train = trainFeature.merge(trainTargetScored, on='sig_id')
test = testFeature.merge(sample_submission, on='sig_id')
#件のレコードを削除。
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test[test['cp_type']!='ctl_vehicle'].reset_index(drop=True)
#cp_typeは使用しない。(今となっては全て同じ特徴量(trt_cp)であるため)
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

target = train[trainTargetScored.columns]

### One-Hot Encoding

In [11]:
feature_name = ['cp_time','cp_dose']
train, test = prp.OneHot_encode(train, test, feature_name)

## Preprocessing Summary

In [12]:
train.head(5)

,sig_id,0,1,2,3,4,5,6,7,8,...,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,cp_time_24,cp_time_48,cp_time_72,cp_dose_D1,cp_dose_D2
0,id_000644bb2,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,...,0,0,0,0,0,1,0,0,1,0
1,id_000779bfc,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,...,0,0,0,0,0,0,0,1,1,0
2,id_000a6266a,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,...,0,0,0,0,0,0,1,0,1,0
3,id_0015fd391,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,...,0,0,0,0,0,0,1,0,1,0
4,id_001626bd3,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,...,0,0,0,0,0,0,0,1,0,1


In [13]:
test.head(5)

,sig_id,0,1,2,3,4,5,6,7,8,...,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,cp_time_24,cp_time_48,cp_time_72,cp_dose_D1,cp_dose_D2
0,id_0004d9e33,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,-0.2140,0.2221,-0.3260,...,0.5,0.5,0.5,0.5,0.5,1,0,0,1,0
1,id_001897cda,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,-1.8820,0.4022,-0.3528,...,0.5,0.5,0.5,0.5,0.5,0,0,1,1,0
2,id_00276f245,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,0.5998,-0.1799,0.9365,...,0.5,0.5,0.5,0.5,0.5,1,0,0,0,1
3,id_0027f1083,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,0.5128,0.6365,0.2611,...,0.5,0.5,0.5,0.5,0.5,0,1,0,1,0
4,id_006fc47b8,0.3658,0.5536,-0.6898,-1.6270,0.5239,-0.3832,-0.4653,1.0070,0.3726,...,0.5,0.5,0.5,0.5,0.5,0,1,0,0,1


In [14]:
target.head(5)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
sample_submission.head(5)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,id_001897cda,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2,id_002429b5b,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3,id_00276f245,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,id_0027f1083,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [16]:
print("Train: "+ str(train.shape))
print("Test: "+ str(test.shape))
print("Target: "+ str(target.shape))
print("sample_submission: "+ str(sample_submission.shape))

Train: (21948, 1139)
Test: (3624, 1139)
Target: (21948, 207)
sample_submission: (3982, 207)


# Dataset Classes

In [17]:
#Train,Valid用のデータクラス
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        #torch.DataLoaderに入れるための形式
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
#Test用のデータクラス
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            #torch.DataLoaderに入れるための形式
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

# Func: Fitting, Evaluation, Predict

In [18]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   
    

# Model architect

In [19]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, param):
        super(Model, self).__init__()
        #hyperoptによる被探索パラメータ
        hidden_size1=param['hidden_size1']
        hidden_size2=param['hidden_size2']
        hidden_size3=param['hidden_size3']
        dropOutRate1=param['dropOutRate1']
        dropOutRate2=param['dropOutRate2']
        #dropOutRate3=param['dropOutRate3']
        #dropOutRate4=param['dropOutRate4']
        
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(dropOutRate1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size1))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size1)
        self.dropout2 = nn.Dropout(dropOutRate1)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size1, hidden_size2))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size2)
        self.dropout3 = nn.Dropout(dropOutRate2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size2, hidden_size3))
        
        self.batch_norm4 = nn.BatchNorm1d(hidden_size3)
        self.dropout4 = nn.Dropout(dropOutRate2)
        self.dense4 = nn.utils.weight_norm(nn.Linear(hidden_size3, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.relu(self.dense3(x))
        
        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.dense4(x)
        
        return x

# Run

## HyperParameter

In [20]:
# HyperParameters
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

## CV folds

In [21]:
folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=NFOLDS)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
    folds.loc[v_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)

In [22]:
folds.head(5)

,sig_id,0,1,2,3,4,5,6,7,8,...,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,cp_time_24,cp_time_48,cp_time_72,cp_dose_D1,cp_dose_D2,kfold
0,id_000644bb2,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,...,0,0,0,0,1,0,0,1,0,0
1,id_000779bfc,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,...,0,0,0,0,0,0,1,1,0,2
2,id_000a6266a,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,...,0,0,0,0,0,1,0,1,0,1
3,id_0015fd391,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,...,0,0,0,0,0,1,0,1,0,2
4,id_001626bd3,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,...,0,0,0,0,0,0,1,0,1,2


## Config about Fitting

In [23]:
#Fitするときに"y"として使う列
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()
#Fitするときに"X"として使う列
#kfold, id等はここで削除。
feature_cols = [c for c in folds.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]

num_features=len(feature_cols)
num_targets=len(target_cols)

## Single Fold Running

In [24]:
def run_training(fold, seed, param):
    
    seed_everything(seed)
    
    train = folds
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        param=param
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    ##### 評価関数 ######
    loss_fn = nn.BCEWithLogitsLoss()
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_fn, trainloader, DEVICE)
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        #print("EPOCH: {:03}: | train_loss: {:.3f}: | valid_loss: {:.3f}".format(epoch, train_loss, valid_loss))
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"{SAVEMODEL}FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            early_step += 1
            if (early_step >= early_stopping_steps):
                print('Early stopping. Best Val loss: {:.3f}'.format(best_loss))
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        param=param
    )
    
    model.load_state_dict(torch.load(f"{SAVEMODEL}FOLD{fold}_.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    
    return oof, predictions


## K-Fold Running

In [25]:
def run_k_fold(NFOLDS, seed, param):
    oof = np.zeros((len(train), num_targets))
    predictions = np.zeros((len(test), num_targets))
    
    for fold in range(NFOLDS):
        #print('=' * 20, 'Fold', fold, '=' * 20)
        oof_, pred_ = run_training(fold, seed, param)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

# Execute

In [26]:
def Exec(param):
    
    # Averaging on multiple SEEDS
    SEED = [0, 1, 2, 3 ,4, 5]
    oof = np.zeros((len(train), num_targets))
    predictions = np.zeros((len(test), num_targets))
    
    for seed in SEED:
        print('~' * 20, 'SEED', seed, '~' * 20)
        oof_, predictions_ = run_k_fold(NFOLDS, seed, param)
        oof += oof_ / len(SEED)
        predictions += predictions_ / len(SEED)
    
    
    #CV score : OOFの評価結果。
    #OOF(学習モデルによるtrain dataの予測)
    train[target_cols] = oof
    #target(答え)
    valid_results = trainTargetScored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
    
    y_true = trainTargetScored[target_cols].values
    y_pred = valid_results[target_cols].values
    
    score = 0
    for i in range(num_targets):
        score_ = log_loss(y_true[:, i], y_pred[:, i]) #問題の評価指標によって変わる。
        score += score_ / target.shape[1]
        
    print("CV log_loss: ", score)
    
    return score, predictions


In [30]:
param_space = {'hidden_size1': 512, 
               'hidden_size2': 512, 
               'hidden_size3': 512, 
               'dropOutRate1': 0.20393004966355735, 
               'dropOutRate2': 0.39170486751620137, 
               #'dropOutRate3': 0.39170486751620137, 
              }
score, predictions = Exec(param_space)

~~~~~~~~~~~~~~~~~~~~ SEED 0 ~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~ SEED 1 ~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~ SEED 2 ~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~ SEED 3 ~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~ SEED 4 ~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~ SEED 5 ~~~~~~~~~~~~~~~~~~~~


In [31]:
print("score: " + str(score))

score: 0.014791937404463365


# Postprocessing & Submit

In [30]:
test[target_cols] = predictions
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv(f'{SUBMIT}submission.csv', index=False)

In [31]:
sub.shape

(3982, 207)

# Hyperparameter Tuning

In [26]:
#hyperopt
from hyperopt import fmin, tpe, hp, rand, Trials

In [27]:
def HOptExec(param):
    
    # Averaging on multiple SEEDS
    SEED = [0, 1, 2, 3 ,4, 5]
    oof = np.zeros((len(train), num_targets))
    predictions = np.zeros((len(test), num_targets))
    
    for seed in SEED:
        #print('~' * 20, 'SEED', seed, '~' * 20)
        oof_, _ = run_k_fold(NFOLDS, seed, param)
        oof += oof_ / len(SEED)
        #predictions += predictions_ / len(SEED)
    
    
    #CV score : OOFの評価結果。
    #OOF(学習モデルによるtrain dataの予測)
    train[target_cols] = oof
    #target(答え)
    valid_results = trainTargetScored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
    
    y_true = trainTargetScored[target_cols].values
    y_pred = valid_results[target_cols].values
    
    score = 0
    for i in range(num_targets):
        score_ = log_loss(y_true[:, i], y_pred[:, i]) #問題の評価指標によって変わる。
        score += score_ / target.shape[1]
        
    #print("CV log_loss: ", score)
    return score
    #return score, predictions


In [28]:
%%time
param_space = {'hidden_size1': hp.choice('hidden_size1', [2048, 1024, 512]), 
               'hidden_size2': hp.choice('hidden_size2', [2048, 1024, 512]), 
               'hidden_size3': hp.choice('hidden_size3', [2048, 1024, 512]), 
               'dropOutRate1': hp.uniform('dropOutRate1', 0.2, 0.6), 
               'dropOutRate2': hp.uniform('dropOutRate2', 0.2, 0.6), 
               #'dropOutRate3': hp.uniform('dropOutRate3', 0.2, 0.6), 
              }

trials = Trials()

hopt = fmin(fn = HOptExec, 
            space = param_space, 
            algo = tpe.suggest, 
            max_evals = 30, 
            #timeout = 8.9 * 60 * 60, 
            trials = trials, 
           )

100%|██████████| 30/30 [5:59:33<00:00, 719.13s/trial, best loss: 0.014635599916671092]  
CPU times: user 6h 37min 46s, sys: 4min 8s, total: 6h 41min 55s
Wall time: 5h 59min 33s


In [29]:
print(hopt)

{'dropOutRate1': 0.20106834415442176, 'dropOutRate2': 0.3576769009996968, 'hidden_size1': 0, 'hidden_size2': 2, 'hidden_size3': 0}


In [ ]:
%%time
param_space = {'hidden_size1': hp.choice('hidden_size1', [2048, 1024, 512]), 
               'hidden_size2': hp.choice('hidden_size2', [2048, 1024, 512]), 
               'hidden_size3': hp.choice('hidden_size3', [2048, 1024, 512]), 
               'dropOutRate1': hp.uniform('dropOutRate1', 0.2, 0.6), 
               'dropOutRate2': hp.uniform('dropOutRate2', 0.2, 0.6), 
               #'dropOutRate3': hp.uniform('dropOutRate3', 0.2, 0.6), 
              }

trials = Trials()

hopt = fmin(fn = HOptExec, 
            space = param_space, 
            algo = tpe.suggest, 
            max_evals = 30, 
            #timeout = 8.9 * 60 * 60, 
            trials = trials, 
           )